In [ ]:
import pandas as pd
import os
from collections import OrderedDict

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

ROOT_PATH = '...'

# Extracting information

In [ ]:
preprocessed_path=ROOT_PATH+'eq_1_demanda_empleo/resources/data/preprocessed/'

# Extracting information from CSV files:
for subdir, dirs, files in os.walk(preprocessed_path):
    df_list=[]
    print('Number of CSV files: ', len(files))
    for file in files:
        file_csv_path=os.path.join(subdir, file)
        # print(file_csv_path)
        preprocessed_data_df  = pd.read_csv(file_csv_path,sep=',',encoding='utf-8',header=None,skiprows=[0])  
        # display(preprocessed_data_df)
        df_list.append(preprocessed_data_df)
    df_data = pd.concat(df_list, axis=0, ignore_index=True)
    df_data.rename({0:'sector',1:'id_job',2:'date',3:'job',4:'city',5:'province',6:'office',7:'content',8:'description',9:'task',10:'requirement',11:'english_level',12:'condition',13:'number'}, axis=1, inplace=True)

print('Number of offers: ', df_data.shape[0])
print('Number of columns: ', df_data.shape[1])

In [ ]:
# Removing duplicated rows:
df_data.drop_duplicates(subset='id_job', keep="last", inplace=True)
print('Number of offers sin duplicados: ', df_data.shape[0])
print('Number of columns: ', df_data.shape[1])

In [ ]:
# Column names:
column_name_list = df_data.columns.tolist()
print('Column names: ', column_name_list)

# Analyzing the extracted information

In [ ]:
# Analyzing unique values by column:
for column_name in column_name_list:
    print('----------------------------------------')
    print('COLUMN NAME: ', column_name)
    unique_data_list = df_data[column_name].unique().tolist()
    print('Number of unique values: ', len(unique_data_list))
    print('Unique values: ', unique_data_list)    

# Preprocessing texts by column

In [ ]:
# Cleaning "province" column:
BAD_PROVINCE_LIST = ['CID Puertomingalvo','MERCAZARAGOZA','MORA DE RUBIELOS Mora De Rubielos']

for bad_province in BAD_PROVINCE_LIST:
    df_data['province']=df_data['province'].astype(str).str.replace(bad_province,'')
    df_data['province']=df_data['province'].astype(str).str.strip()
    
df_data['province']=df_data['province'].astype(str).str.replace('HUSCA','HUESCA')
df_data['province']=df_data['province'].astype(str).str.replace('Zaragoza','ZARAGOZA')

print('Unique values: ', df_data['province'].unique().tolist())
df_province = df_data.groupby(['province']).size().reset_index(name='count')
display(df_province)

In [ ]:
# Cleaning "city" column:
BAD_LOCATION_LIST = ['S-MARMITONES', 'S-AJUSTADORES DEL AUTOMÓVIL, EN GENERAL.', 'A-MÓVIL', 'A-MOVIL', 'S-OPERADORES DE HORMIGONERA', 'S-OPERADORES DE GRÚA EN CAMIÓN.', 'S-REPARADORES DE PALETS DE MADERA', 'A- OFICIAL DE PRIMERA', '-EMPLEO PÚBLICO-', 'C@S', 'S-PINTORES DE VEHÍCULOS', '- AJUSTADOR/A DEL AUTOMÓVIL', 'A-ELECTRICISTA Y/O ELECTRÓNICO/A DE AUTOMOCIÓN', 'S-AYUDANTES-FREGAPLATOS', 'A-TELEFONISTA', 'R@S Y AYUDANTES DE COCINA', 'R@', 'S-REPOSTEROS', 'N-CARRETILLERO', 'I@ - TRABAJADOR/A APÍCOLA', '- DEPENDIENTE DE FRUTERÍA', 'A-CARNICERO/A', 'A-CAFETERÍA', 'I@', '- TRABAJADOR/A APÍCOLA', 'S-ELECTRICISTAS', 'A-DEPENDIENTE/A INFORMÁTICA', '- RECAMBISTAS', 'C@ MANTENIMIENTO', 'O-COMERCIALES', '-EMPLEO PÚBLICO-', 'G-MAG, TORNEROS-FRESADORES, MECÁNICOS DE MAQUINARIA PESADA, MECATRÓNICOS, ELECTRICISTAS DE AUTOMOCIÓN', 'S-CRISTALEROS', 'G-MAG', 'N-REPARTO MERCANCIA']
BAD_LOCATION_TUPLE_LIST = [('Maria de Huerva','María de Huerva'),('Villanueva de G‡llego','Villamayor de Gállego'),('ƒpila','Épila'),('Alca–iz', 'Alcañiz'), ('A’nsa-Sobrarbe', 'Aínsa-Sobrarbe'), ('BinŽfar', 'Binéfar'), ('Gra–Žn', 'Grañén'), ('H’jar', 'Híjar'), ('La Puebla de AlfindŽn', 'La Puebla de Alfindén'), ('Monz—n', 'Monzón'), ('Sabi–‡nigo', 'Sabiñánigo'), ('Sallent de G‡llego', 'Sallent de Gállego'), ('Villamayor de G‡llego', 'Villamayor de Gállego'), ('Andorra', 'Andorra, Teruel'), ('Andorra, Teruel, Teruel', 'Andorra, Teruel'), ('Alfajar’n', 'Alfajarín'), ('Mazale—n', 'Mazaleón'), ('Caba–as de Ebro', 'Cabañas de Ebro'), ('ATECA', 'Ateca'), ('BARBASTRO', 'Barbastro'), ('HUESCA', 'Huesca'), ('MONZîN', 'Monzón'), ('Sari–ena', 'Sariñena'), ('TARAZONA', 'Tarazona'), ('TERUEL', 'Teruel'), ('ZARAGOZA', 'Zaragoza')]

for bad_location in BAD_LOCATION_LIST:
    df_data['city']=df_data['city'].astype(str).str.replace(bad_location,'')
    
for bad_location_tuple in BAD_LOCATION_TUPLE_LIST:
    #df_data['city'].astype(str).str
    #print(df_data['city'].astype(str).str)
    #df_data['city']=' '.join(bad_location_tuple[1] if word == bad_location_tuple[0] else word for word in df_data['city'].astype(str).str.split())
    df_data['city']=df_data['city'].astype(str).str.replace(bad_location_tuple[0],bad_location_tuple[1])    
    df_data['city']=df_data['city'].astype(str).str.strip()

df_data['city']=df_data['city'].astype(str).str.replace('A-ORDESA','Ordesa')

print('Unique values: ', df_data['city'].unique().tolist())
df_city = df_data.groupby(['city']).size().reset_index(name='count')
display(df_city)

In [ ]:
# Cleaning "office" column:
BAD_OFFICE_LIST = [('Sabi–‡nigo', 'SABIÑANIGO'), ('SABI„ANIGO', 'SABIÑANIGO'), ('Alca–iz', 'ALCAÑIZ'), ('MONZON', 'MONZÓN'), ('CASPE', 'ZARAGOZA-COMPROMISO DE CASPE'), ('ZARAGOZA-COMPROMISO DE', 'ZARAGOZA-COMPROMISO DE CASPE'), ('EJEA', 'EJEA DE LOS CABALLEROS'), ('ANDORRA', 'ANDORRA, TERUEL'), ('ZARAGORA-CENTRO', 'ZARAGOZA-CENTRO'),('ANDORRA, TERUEL, TERUEL','ANDORRA, TERUEL')]
for bad_office in BAD_OFFICE_LIST:
    df_data['office']=df_data['office'].astype(str).str.replace(bad_office[0], bad_office[1])
    df_data['office']=(df_data['office'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
    df_data['office']=df_data['office'].astype(str).str.strip()

print('Unique values: ', df_data['office'].unique().tolist())
df_city = df_data.groupby(['office']).size().reset_index(name='count')
display(df_city)

# Saving cleaned data

In [ ]:
cleaned_path=ROOT_PATH+'eq_1_demanda_empleo/resources/data/cleaned/df_data.csv'

df_data.to_csv(cleaned_path, index=False)